# BJJ 


In [1]:
import urllib.request, urllib.parse, urllib.error
from bs4 import BeautifulSoup
import ssl
import pandas as pd
import numpy as np
import re
import sqlite3

In [2]:
# Get all athletes
url = 'https://www.flograppling.com/people'
html = urllib.request.urlopen(url).read()
soup = BeautifulSoup(html, 'html.parser')
#print(soup)

In [3]:
Athletes = pd.DataFrame(columns=['Athlete_Name','Athlete_ID'], index=range(0,1000))

row = 0
for t in soup.find_all('div', class_ = 'grid-container-item grid-container-item__min-width mb-4 ng-star-inserted'):
    Name = t.get_text()
    Athletes.iloc[row,0] = Name
    for p in t('a'):
        ID = p.get('href')
        Athletes.iloc[row,1] = ID
        row = row + 1
Athletes = Athletes.dropna(axis=0,how='all')
Athletes

,Athlete_Name,Athlete_ID
0,"Aaron ""Tex"" Johnson",/people/6600946-aaron-tex-johnson
1,Adam Wardzinski,/people/5950131-adam-wardzinski
2,Alexssandro Sodre,/people/6749113-alexssandro-sodre
3,Ana Carolina Vieira,/people/5950497-ana-carolina-vieira
4,Andre Galvao,/people/5951184-andre-galvao
...,...,...
97,Vagner Rocha,/people/5950392-vagner-rocha
98,Victor Hugo,/people/6566365-victor-hugo
99,"Vinicius ""Trator"" Ferreira",/people/6708867-vinicius-trator-ferreira
100,Vinny Magalhaes,/people/5950917-vinny-magalhaes


# Can I get current record & Age

In [29]:
# Sample Athlete
#url = 'https://www.flograppling.com/people/6566775-craig-jones'
#url = 'https://www.flograppling.com/people/6801756-danielle-kelly'
#url = 'https://www.flograppling.com/people/6801763-haisam-rida'
#url = 'https://www.flograppling.com/people/6578323-cole-abate'
url = 'https://www.flograppling.com/people/6566375-ethan-crelinsten'
html = urllib.request.urlopen(url).read()
soup = BeautifulSoup(html, 'html.parser')

born_pattern = r'born(?: on)? (\w+ \d{1,2}(?:st|nd|rd|th)?, \d{4})'

#soup
#print(soup.find('script id="structuredSchema"'))
for line in soup('meta'):
    #print(str(line.get('content')))
    textline = line.get('content')
    if textline is not None:
        if 'born' in textline:
            born_match=re.search(born_pattern, textline,re.IGNORECASE)
            birthdate = born_match.group(1)
            print(birthdate)
            print('----')
# Current Record
try:
    record_text = soup('h2')[0].get_text()
    print(record_text.split()[0])
    record_num=record_text.split()[0]
except:
    print('No Record')

37-15-1


In [36]:
def get_record(athlete_parsed_html):
    try:
        current_record_text = athlete_parsed_html('h2')[0].get_text()
        #current_record_text = athlete_parsed_html('h2')[0].get_text()
        current_record_num = current_record_text.split()[0]
        return(current_record_num)
    except:
        print('No Record')   

In [37]:
get_record(soup)

'37-15-1'

In [32]:
def get_age(athlete_parsed_html):
    born_pattern = r'born(?: on)? (\w+ \d{1,2}(?:st|nd|rd|th)?, \d{4})'
    for line in athlete_parsed_html('meta'):
        textline = line.get('content')
        if textline is not None:
            if 'born' in textline:
                born_match=re.search(born_pattern, textline,re.IGNORECASE)
                birthdate = born_match.group(1)
                birthdate = birthdate.replace(',','')
                return birthdate

In [33]:
get_age(soup)

In [68]:
for line in soup('meta'):
    textline = line.get('content')
    if textline is not None:
        # Convert to Lowercase
        textline = textline.lower()
        #print(textline.split())
        # find "born"
        #print(textline.find('born'))
        #print(textline[75:79])
        if textline.find('born') != -1:
            if textline.find('born on') != -1:
                post_born = textline[textline.find('born on'):]
                split_string = post_born.split()
                #print(split_string[2:5])
                birthdate = (split_string[2] + ' ' + split_string[3] + ' ' + split_string[4])
                birthdate = birthdate.replace(',', '')
                print(birthdate)
                #print(post_born)
                break
            else:
                post_born = textline[textline.find('born'):]
                split_string = post_born.split()
                split_string = post_born.split()
                #print(split_string[1:4])
                birthdate = (split_string[1] + ' ' + split_string[2] + ' ' + split_string[3])
                birthdate = birthdate.replace(',', '')
                print(birthdate)
                #print(post_born)
                break

may 14 1994


In [2]:
# Connect to SQLite Server
con = sqlite3.connect("bjj_network.db")
cur = con.cursor()

In [4]:
# Create a table for athletes
# cur.execute("CREATE TABLE Athlete_List(Athlete_Name, Athlete_ID, DOB, Current_Record)")

OperationalError: table Athlete_List already exists

In [6]:
cur.execute("SELECT DOB FROM Athlete_List").fetchone()

# Table Format
0. Year
1. Date
2. W/L
3. Replay
4. Method
5. Opponent
6. Weight
7. Event

In [150]:
# Sample Athlete
url = 'https://www.flograppling.com/people/6566775-craig-jones'
html = urllib.request.urlopen(url).read()
soup = BeautifulSoup(html, 'html.parser')
# Loop
Columns = ['Year','Date','Date_Link','W/L','W/L_Link','Replay','Replay_Link','Method','Method_Link','Opponent','Opponent_Link','Weight','Weight_Link','Event','Event_Link']
Athlete_Matches = pd.DataFrame(columns=Columns,index=range(0,1000)) # Premake table
idx=1
rownum = 0
for t in soup('flo-section-table'): # This will get all the tables on the page
    
    Year = t.find('h5').get_text()
    if Year == 'W/L Methods': # Flo has a table that we don't care about called 'W/L Methods'
        break
    #print(t.find('h6'))
    #print('-------') 
    for td in t.find_all('td'): # td is cell info within a table in html
        link_idx = idx + 1 
        #print(td.get_text())
        text = td.get_text() # the text stored in the cell (what is visible on the website)
        if text == 'Submission': # This line is meant to stop the loop at the end of the tables
            break # Note that Flo has a table with win/loss methods & submission win/loss methods at the end
        link = td('a')[0].get('href') # the link associated with that text (if there is one)

        
        Athlete_Matches.iloc[rownum, idx] = text
        Athlete_Matches.iloc[rownum, link_idx] = link
        #print('Column:',idx,'Year:',Year,'Text:',text,'Link:',link) # Print it all so I can check it works
        idx = idx + 2
        
        if idx == 15: # This is the piece that keeps track of what column it is
            idx=1
            Athlete_Matches.iloc[rownum, 0] = Year # Add year right before we switch the row
            rownum = rownum + 1
    #for piece in t:
        #print(piece.find_all('td'))
Athlete_Matches = Athlete_Matches.dropna(axis=0,how='all') # Get rid of all the extra rows
display(Athlete_Matches)


,Year,Date,Date_Link,W/L,W/L_Link,Replay,Replay_Link,Method,Method_Link,Opponent,Opponent_Link,Weight,Weight_Link,Event,Event_Link
0,2022,09/17,,L,,Replay,/events/7096389/videos?playing=7975476,Points,/events/7096389/videos?playing=7975476,K. Duarte,/people/6451325,-99kgs,,2022 ADCC World Championships,/events/7096389
1,2022,09/17,,W,,Replay,/events/7096389/videos?playing=7975414,Decision,/events/7096389/videos?playing=7975414,N. Meregali,/people/5950129,-99kgs,,2022 ADCC World Championships,/events/7096389
2,2022,09/17,,W,,Replay,/events/7096389/videos?playing=7974649,Guillotine,/events/7096389/videos?playing=7974649,K. Boehm,/people/6600891,-99kgs,,2022 ADCC World Championships,/events/7096389
3,2022,09/17,,W,,Replay,/events/7096389/videos?playing=7974406,Shoulder Lock,/events/7096389/videos?playing=7974406,J. Costa,/people/6808810,-99kgs,,2022 ADCC World Championships,/events/7096389
4,2022,08/27,,W,,,,RNC,,E. Savidov,,Absolute Division,,2022 AIGA Arlan Grip Pro 18,/events/10668133
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,2014,03/12,,W,,,,Inside Heel Hook,,T. Spriggs,/people/5950976,---,,2014 Pan Jiu-Jitsu IBJJF Championship,
73,2014,03/12,,D,,,,N/A - Draw,,S. Grecicho,,---,,2014 Pan Jiu-Jitsu IBJJF Championship,
74,2014,03/12,,W,,,,Leg Lock,,M. Verzbickas,,---,,2014 Pan Jiu-Jitsu IBJJF Championship,
75,2014,03/12,,L,,,,Kimura,,R. Barral,/people/5947295,-88kgs,,2014 Pan Jiu-Jitsu IBJJF Championship,


In [33]:
soup

<!DOCTYPE html>
<html lang="en"><head>
<script src="https://cmp.osano.com/AzyWAQS5NWEEWkU9/55c463c7-97ce-4e58-9403-bf5693e6fd09/osano.js?language=en"></script>
<script src="/osano-flo.js"></script>
<!-- 1plusX -->
<script>
      (function(o,n,e,p,l,u,s){o[l]=o[l]||function(){(o[l].q=o[l].q||[]).push(arguments);};
        u=n.createElement(e);u.async=1;u.src=p;s=n.getElementsByTagName(e)[0];s.parentNode.insertBefore(u,s);
      }(window,document,"script","https://cdn.opecloud.com/ope-pdmp.js","ope"));
      ope("pdmp", "init", "b165", "auto");
    </script>
<!-- end 1plusX -->
<script>window.grumi = {
      cfg: {
        advs: {
          '4409051465': true,
          '4739813436': true,
          '5007374826': true,
          '4995583385': true
        },
        pbGlobal: 'owpbjs'
      },
      key: '33cea31c-ec69-44af-afb7-3e0656e35da6'
    };</script>
<script async="" src="//rumcdn.geoedge.be/33cea31c-ec69-44af-afb7-3e0656e35da6/grumi-ip.js"></script>
<!-- auth -->
<script async="

['55-20-2', '(W-L-D)']
